# Airbyte CDK (已弃用)

注意：`AirbyteCDKLoader` 已弃用。请改用 [`AirbyteLoader`](/docs/integrations/document_loaders/airbyte)。

>[Airbyte](https://github.com/airbytehq/airbyte) 是一个数据集成平台，用于从 API、数据库和文件到数据仓库和湖的数据 ELT 管道。它拥有最大的数据仓库和数据库 ELT 连接器目录。

许多源连接器是使用 [Airbyte CDK](https://docs.airbyte.com/connector-development/cdk-python/) 实现的。此加载器允许运行这些连接器中的任何一个，并将数据作为文档返回。

## 安装

首先，你需要安装 `airbyte-cdk` Python 包。

In [ ]:
%pip install --upgrade --quiet  airbyte-cdk

然后，你可以从 [Airbyte Github 仓库](https://github.com/airbytehq/airbyte/tree/master/airbyte-integrations/connectors) 安装一个现有的连接器，或者使用 [Airbyte CDK](https://docs.airbyte.io/connector-development/connector-development) 创建你自己的连接器。

例如，要安装 Github 连接器，请运行

In [ ]:
%pip install --upgrade --quiet  "source_github@git+https://github.com/airbytehq/airbyte.git@master#subdirectory=airbyte-integrations/connectors/source-github"

一些来源也以常规包的形式发布在 PyPI 上

## 示例

现在，你可以根据导入的源创建一个 `AirbyteCDKLoader`。它需要一个传递给连接器的 `config` 对象。你还需要通过名称（`stream_name`）选择要从中检索记录的流。有关 `config` 对象和可用流的更多信息，请查看连接器文档页面和 spec 定义。对于 Github 连接器，它们是：

* [https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-github/source_github/spec.json](https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-github/source_github/spec.json)
* [https://docs.airbyte.com/integrations/sources/github/](https://docs.airbyte.com/integrations/sources/github/)

In [ ]:
from langchain_community.document_loaders.airbyte import AirbyteCDKLoader
from source_github.source import SourceGithub  # plug in your own source here

config = {
    # your github configuration
    "credentials": {"api_url": "api.github.com", "personal_access_token": "<token>"},
    "repository": "<repo>",
    "start_date": "<date from which to start retrieving records from in ISO format, e.g. 2020-10-20T00:00:00Z>",
}

issues_loader = AirbyteCDKLoader(
    source_class=SourceGithub, config=config, stream_name="issues"
)

现在您可以通过常规方式加载文档

In [ ]:
docs = issues_loader.load()

由于 `load` 返回一个列表，它将一直阻塞直到所有文档都被加载。为了更好地控制此过程，您也可以使用 `lazy_load` 方法，它会返回一个迭代器：

In [ ]:
docs_iterator = issues_loader.lazy_load()

请注意，默认情况下页面内容为空，并且 metadata 对象包含记录中的所有信息。要以不同的方式创建文档，请在创建加载器时传入一个 `record_handler` 函数：

In [ ]:
from langchain_core.documents import Document


def handle_record(record, id):
    return Document(
        page_content=record.data["title"] + "\n" + (record.data["body"] or ""),
        metadata=record.data,
    )


issues_loader = AirbyteCDKLoader(
    source_class=SourceGithub,
    config=config,
    stream_name="issues",
    record_handler=handle_record,
)

docs = issues_loader.load()

## 增量加载

某些流支持增量加载，这意味着源会跟踪已同步的记录，而不会再次加载它们。这对于数据量大且更新频繁的源非常有用。

为了利用这一点，请存储加载器的 `last_state` 属性，并在再次创建加载器时将其传入。这将确保只加载新记录。

In [ ]:
last_state = issues_loader.last_state  # store safely

incremental_issue_loader = AirbyteCDKLoader(
    source_class=SourceGithub, config=config, stream_name="issues", state=last_state
)

new_docs = incremental_issue_loader.load()